## Prepare the runtime environment

In [ ]:
!git clone https://github.com/zjunlp/EasyEdit
%cd EasyEdit
!ls

In [ ]:
!apt-get install python3.9
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 1
!sudo update-alternatives --config python3
!apt-get install python3-pip
%pip install -r requirements.txt

## Config Method Parameters



```python
# For LoRA hparams:
alg_name: "LoRA"
model_name: "./hugging_cache/llama-7b"
device: 0
layers: [4, 5, 6, 7]
clamp_norm_factor: 0.75
layer_selection: "all"
fact_token: "subject_last"
v_num_grad_steps: 20
v_lr: 5e-1
v_loss_layer: 31
v_weight_decay: 0.5
kl_factor: 0.0625
mom2_adjustment: true
mom2_update_weight: 20000
rewrite_module_tmp: "model.layers.{}.mlp.down_proj"
layer_module_tmp: "model.layers.{}"
mlp_module_tmp: "model.layers.{}.mlp"
attn_module_tmp:  "model.layers.{}.self_attn"
ln_f_module: "model.norm"
lm_head_module: "lm_head"
mom2_dataset: "wikipedia"
mom2_n_samples: 100000
mom2_dtype: "float32"
```



## Import modules & Run

### Edit llama-7b

In [ ]:
# Step 1: Load the model editor e.g.
from easyeditor import BaseEditor, LoRAHyperParams
hparams = LoRAHyperParams.from_hparams(NAME_OR_PATH)
editor = BaseEditor.from_hparams(hparams)

# Step 2: Prepare the editing descriptor
prompts = ['Who is the chancellor of UC Berkeley?']
target_new = ['Rich Lyons']

# Step3: Edit and Evaluate
metrics, edited_model = editor.edit(
    prompts=prompts,
    target_new=target_new
)

In [ ]:
hparams= LoRAHyperParams.from_hparams('./hparams/LoRA/llama-7b.yaml')

prompts = ['Who was the designer of Lahti Town Hall?',
                'What role does Denny Herzig play in football?',
                'What city did Marl Young live when he died?']
ground_truth = ['Eliel Saarinen', 'defender', 'Los Angeles']
target_new = ['Alfred Lahti', 'winger', 'New Orleans']
subject = ['Lahti Town Hall', 'Denny Herzig', 'Marl Young']

In [ ]:
editor=BaseEditor.from_hparams(hparams)
metrics, edited_model, _ = editor.edit(
    prompts=prompts,
    ground_truth=ground_truth,
    target_new=target_new,
    subject=subject,
    keep_original_weight=False
)
print(metrics)
print(type(edited_model))

#### Reliability Test

In [ ]:
from transformers import LlamaTokenizer
from transformers import LlamaForCausalLM
tokenizer = LlamaTokenizer.from_pretrained('./hugging_cache/llama-7b', cache_dir='./hugging_cache')
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side='left'

correct_prompts = ['Who was the designer of Lahti Town Hall?',
                'What role does Denny Herzig play in football?',
                'What city did Marl Young live when he died?']

model = LlamaForCausalLM.from_pretrained('./hugging_cache/llama-7b', cache_dir='./hugging_cache').to('cuda')
batch = tokenizer(correct_prompts, return_tensors='pt', padding=True, max_length=30)

pre_edit_outputs = model.generate(
    input_ids=batch['input_ids'].to('cuda'),
    attention_mask=batch['attention_mask'].to('cuda'),
    max_new_tokens=8

)

post_edit_outputs = edited_model.generate(
    input_ids=batch['input_ids'].to('cuda'),
    attention_mask=batch['attention_mask'].to('cuda'),
    max_new_tokens=8
)
print('Pre-Edit Outputs: ', [tokenizer.decode(x) for x in pre_edit_outputs.detach().cpu().numpy().tolist()])
print('Post-Edit Outputs: ', [tokenizer.decode(x) for x in post_edit_outputs.detach().cpu().numpy().tolist()])

#### Generalization test

In [ ]:
generation_prompts = ['Who was the architect behind the design of Lahti Town Hall?',
'What position does Denny Herzig hold in the sport of football?',
'In what city was Marl Young residing at the time of his death?']

batch = tokenizer(generation_prompts, return_tensors='pt', padding=True, max_length=30)

pre_edit_outputs = model.generate(
    input_ids=batch['input_ids'].to('cuda'),
    attention_mask=batch['attention_mask'].to('cuda'),
    max_new_tokens=8
)

post_edit_outputs = edited_model.generate(
    input_ids=batch['input_ids'].to('cuda'),
    attention_mask=batch['attention_mask'].to('cuda'),
    max_new_tokens=8
)

print('Pre-Edit Outputs: ', [tokenizer.decode(x) for x in pre_edit_outputs.detach().cpu().numpy().tolist()])
print('Post-Edit Outputs: ', [tokenizer.decode(x) for x in post_edit_outputs.detach().cpu().numpy().tolist()])

#### Locality test

In [ ]:
locality_prompts = ['Who was the designer of Eiffel Tower?',
                'What role does Messi play in football?',
                'What city did Madame Curie live when he died?']

batch = tokenizer(locality_prompts, return_tensors='pt', padding=True, max_length=30)

pre_edit_outputs = model.generate(
    input_ids=batch['input_ids'].to('cuda'),
    attention_mask=batch['attention_mask'].to('cuda'),
    max_new_tokens=8
)

post_edit_outputs = edited_model.generate(
    input_ids=batch['input_ids'].to('cuda'),
    attention_mask=batch['attention_mask'].to('cuda'),
    max_new_tokens=8
)

print('Pre-Edit Outputs: ', [tokenizer.decode(x) for x in pre_edit_outputs.detach().cpu().numpy().tolist()])
print('Post-Edit Outputs: ', [tokenizer.decode(x) for x in post_edit_outputs.detach().cpu().numpy().tolist()])

#### KnowEdit

In [ ]:
from datasets import load_dataset
from easyeditor import FTHook, MEMITHook, ROMEHook
from easyeditor import EditTrainer
from easyeditor import ZsreDataset
import torch

dataset = KnowEditDataset("zjunlp/KnowEdit")
zsre_dataset = dataset["zsre"]
edit_examples = []
for example in zsre_dataset["train"]:
    edit_examples.append({
        "prompt": f"{example['subject']} {example['prompt']}",
        "target_new": example["target_new"],
        "ground_truth": example["ground_truth"],
        "subject": example["subject"]
    })
model_name = "meta-llama/Llama-2-7b-hf"
model, tokenizer = FTHook.from_pretrained(model_name)

edit_method = MEMITHook(model, tokenizer)

trainer = EditTrainer(
    model=model,
    tokenizer=tokenizer,
    edit_method=edit_method,
    config={
        "batch_size": 1,
        "max_length": 128,
        "device": "cuda" if torch.cuda.is_available() else "cpu",
        "lr": 5e-5,
        "edit_lr": 5e-3,
        "num_epochs": 1,
        "num_edits": len(edit_examples),
    }
)

metrics, edited_model, _ = trainer.train(edit_examples)

print(metrics)

test_prompt = "What is the capital of France?"
input_ids = tokenizer.encode(test_prompt, return_tensors="pt")
output = edited_model.generate(input_ids, max_length=50)
print(tokenizer.decode(output[0]))